In [1]:
import re #pacchetto per espressioni regolari
import os #pacchetto per muoversi nelle cartelle
import sys
sys.path.append('..')  # Add parent directory to the Python path
from utils.helpers import *


In [2]:
train_path = 'data/training.txt'
test_path = 'data/test.txt'

out_dir = 'data/profiling_input'

In [3]:
split = get_split(train_path)
split

'training'

In [5]:

# Necessario eitchettare il blind test

# Read gold.txt file and extract gender values
with open("data/gold.txt", "r") as gold_file:
    gold_data = gold_file.readlines()

# Extract gender labels
gender_list = [line.split()[1].strip() for line in gold_data]

# Read test.txt file
with open("data/GxG_Twitter (1).txt", "r") as test_file:
    test_data = test_file.readlines()

# Replace "?" with gender labels sequentially
updated_test_data = []
gender_index = 0  # Index to track which gender label to use
in_doc = False
for line in test_data:
    if '<doc id="' in line:
        in_doc = True
        if '?' in line:
            label = gender_list[gender_index]
            updated_line = line.replace("?", label)
            gender_index += 1
        else:
            updated_line = line
    elif "</doc>" in line:
        in_doc = False
        updated_line = line
    elif in_doc:
        # If inside a <doc>...</doc> block, continue appending the lines
        updated_line = line.replace("?", label)
    else:
        updated_line = line
    updated_test_data.append(updated_line)

# Write the updated test data to a new file
with open("data/test.txt", "w") as updated_test_file:
    updated_test_file.writelines(updated_test_data)


In [ ]:
import os

for file_path in [train_path, test_path]:
    split = get_split(file_path)

    for line in open(file_path, 'r'):
        line = line.strip() # remove any leading/trailing whitespace
        
        if '<doc id="' in line:    # find a new document
            doc_id, genre, gender = get_doc_info(line)
            out_file_name = f'{split}#{doc_id}#{genre}#{gender}.txt'
            # Replace "?" with "Question Punct. Mark" in the output file name. "?" gives os error
            out_file_name = out_file_name.replace("?", "QPM")
            out_file_path = os.path.join(out_dir, out_file_name)
            user_post = []

        elif '</doc>' in line:     # end of user
            with open(out_file_path, 'w') as out_file:
                for post in user_post:
                    out_file.write(post)
                    out_file.write('\n')

        elif line != '':  # text of a post
            user_post.append(line)
